In [6]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.colors import Normalize
import glob
import os

In [7]:
nc_files = sorted( glob.glob( r"C:\Users\locfa_v05v5qi\Documents\escadas_termohalinas\raw\Dataset\**\*.nc", recursive=True ) )


In [8]:
def subset_region(ds):
    # Ajustar longitudes se estiverem em 0–360
    if ds.lon.max() > 180:
        ds = ds.assign_coords(
            lon=((ds.lon + 180) % 360) - 180
        )

    # Criar máscara apenas para a dimensão n (perfis)
    mask = (
        (ds.lat <= 10) & (ds.lat >= -60) &
        (ds.lon >= -50) & (ds.lon <= -10)
    )

    if "Nobs" in ds.dims:
        ds = ds.isel(Nobs=mask)

    return ds
    
def check_staircase_exists(ml_h, gl_h): #verifica se os valores são fisicamente válidos (n só se existem)

    valid_ml = np.sum(~np.isnan(ml_h) & (ml_h > 0))
    valid_gl = np.sum(~np.isnan(gl_h) & (gl_h > 0))

    return (valid_ml >= 1) and (valid_gl >= 1)
    
def detect_staircases(ds):

    profile_dim = ds.lat.dims[0]
    n_profiles = ds.sizes[profile_dim] #p n ter problema d confundir "n" e "Nobs"

    staircase_sf = np.zeros(n_profiles, dtype=bool)
    staircase_dc = np.zeros(n_profiles, dtype=bool)

    for i in range(n_profiles):

        # =========================
        # SALT FINGER
        # =========================
        ml_mask_sf = ds.mask_ml_sf_layer.isel({profile_dim: i}) > 0
        gl_mask_sf = ds.mask_gl_sf_layer.isel({profile_dim: i}) > 0

        ml_h_sf = ds.ml_h.isel({profile_dim: i}).values[ml_mask_sf.values] #pega os valores onde ml_mask_sf é True 
        gl_h_sf = ds.gl_h.isel({profile_dim: i}).values[gl_mask_sf.values]

        if len(ml_h_sf) > 0 and len(gl_h_sf) > 0:
            staircase_sf[i] = check_staircase_exists(
                ml_h_sf, gl_h_sf
            )

        # =========================
        # DIFFUSIVE CONVECTION
        # =========================
        ml_mask_dc = ds.mask_ml_dc_layer.isel({profile_dim: i}) > 0
        gl_mask_dc = ds.mask_gl_dc_layer.isel({profile_dim: i}) > 0

        ml_h_dc = ds.ml_h.isel({profile_dim: i}).values[ml_mask_dc.values]
        gl_h_dc = ds.gl_h.isel({profile_dim: i}).values[gl_mask_dc.values]

        if len(ml_h_dc) > 0 and len(gl_h_dc) > 0:
            staircase_dc[i] = check_staircase_exists(
                ml_h_dc, gl_h_dc
            )

    ds["staircase_sf"] = (profile_dim, staircase_sf)
    ds["staircase_dc"] = (profile_dim, staircase_dc)

    return ds

In [9]:
import numpy as np

def diagnose_coordinates(ds, profile_dim="Nobs"):

    print("\n================ DIAGNÓSTICO ESPACIAL ================")

    lat = ds.lat.values
    lon = ds.lon.values
    n = len(lat)

    print("Total de perfis analisados:", n)

    # ----------------------------------------------------
    # 1️⃣ Verificar NaN / Inf
    # ----------------------------------------------------
    nan_lat = np.isnan(lat)
    nan_lon = np.isnan(lon)
    inf_lat = np.isinf(lat)
    inf_lon = np.isinf(lon)

    print("\n--- Valores inválidos ---")
    print("Lat NaN:", np.sum(nan_lat))
    print("Lon NaN:", np.sum(nan_lon))
    print("Lat Inf:", np.sum(inf_lat))
    print("Lon Inf:", np.sum(inf_lon))

    invalid_idx = np.where(nan_lat | nan_lon | inf_lat | inf_lon)[0]
    if len(invalid_idx) > 0:
        print("Perfis com coordenadas inválidas (primeiros 10):", invalid_idx[:10])

    # ----------------------------------------------------
    # 2️⃣ Range esperado
    # ----------------------------------------------------
    print("\n--- Faixa de valores ---")
    print("Lat min/max:", np.nanmin(lat), "/", np.nanmax(lat))
    print("Lon min/max:", np.nanmin(lon), "/", np.nanmax(lon))

    if np.nanmax(lat) > 90 or np.nanmin(lat) < -90:
        print("⚠️ Latitude fora do intervalo físico (-90, 90)")

    if np.nanmax(lon) > 360 or np.nanmin(lon) < -180:
        print("⚠️ Longitude fora do intervalo esperado")

    # ----------------------------------------------------
    # 3️⃣ Sistema de longitude
    # ----------------------------------------------------
    if np.nanmax(lon) > 180:
        print("🌍 Longitude parece estar em 0–360")
    else:
        print("🌍 Longitude parece estar em -180–180")

    # ----------------------------------------------------
    # 4️⃣ Coordenadas duplicadas exatas
    # ----------------------------------------------------
    coords = list(zip(lat, lon))
    unique_coords = set(coords)

    print("\n--- Repetição exata ---")
    print("Total pontos:", len(coords))
    print("Pontos únicos:", len(unique_coords))
    print("Repetidos exatos:", len(coords) - len(unique_coords))

    # ----------------------------------------------------
    # 5️⃣ Resolução real das coordenadas
    # ----------------------------------------------------
    lat_sorted = np.sort(np.unique(lat[~np.isnan(lat)]))
    lon_sorted = np.sort(np.unique(lon[~np.isnan(lon)]))

    lat_diff = np.diff(lat_sorted)
    lon_diff = np.diff(lon_sorted)

    print("\n--- Resolução espacial ---")

    if len(lat_diff) > 0:
        print("Menor Δlat:", np.min(lat_diff))
        print("Mediana Δlat:", np.median(lat_diff))

    if len(lon_diff) > 0:
        print("Menor Δlon:", np.min(lon_diff))
        print("Mediana Δlon:", np.median(lon_diff))

    # ----------------------------------------------------
    # 6️⃣ Saltos anômalos
    # ----------------------------------------------------
    print("\n--- Saltos espaciais entre perfis consecutivos ---")

    dlat = np.abs(np.diff(lat))
    dlon = np.abs(np.diff(lon))

    big_jump = np.where((dlat > 10) | (dlon > 10))[0]

    print("Saltos >10° encontrados:", len(big_jump))

    if len(big_jump) > 0:
        print("Perfis com salto grande (primeiros 10):", big_jump[:10])

    # ----------------------------------------------------
    # 7️⃣ Sensibilidade ao arredondamento
    # ----------------------------------------------------
    print("\n--- Sensibilidade ao arredondamento ---")

    for decimals in [4, 3, 2, 1]:
        lat_r = np.round(lat, decimals)
        lon_r = np.round(lon, decimals)
        n_unique = len(set(zip(lat_r, lon_r)))
        print(f"{decimals} casas decimais → {n_unique} pontos únicos")

    print("\n================ FIM DO DIAGNÓSTICO ==================\n")

In [10]:
# =========================================================
# CONTADORES GERAIS
# =========================================================

total_profiles = 0
total_profiles_region = 0
total_profiles_stair = 0

total_profiles_sf = 0
total_profiles_dc = 0
total_profiles_mixed = 0

# Estatísticas verticais
all_ml_depth_sf = []
all_ml_depth_dc = []

all_ml_thickness_sf = []
all_ml_thickness_dc = []

n_layers_sf = []
n_layers_dc = []

# Pontos espaciais
all_points = []
all_points_stair = []
all_points_sf = []
all_points_dc = []
all_points_mixed = []

results = []

# =========================================================
# LOOP PRINCIPAL
# =========================================================

for file in nc_files:

    print(f"\nProcessando: {os.path.basename(file)}")

    ds = xr.open_dataset(file)

    # ---------------- PERFIS GLOBAIS ----------------
    n_global = ds.sizes.get("Nobs", 0)
    total_profiles += n_global
    print("Perfis globais:", n_global)

    # ---------------- RECORTE ----------------
    ds = subset_region(ds)
    
    n_region = ds.sizes.get("Nobs", 0)
    total_profiles_region += n_region
    print("Perfis na região:", n_region)
    
    if ds.sizes.get("Nobs", 0) > 0:
        diagnose_coordinates(ds)
    
    if n_region == 0:
        continue

    # ---------------- DETECÇÃO ----------------
    ds = detect_staircases(ds)

    sf_mask = ds.staircase_sf #array booleano por perfil
    dc_mask = ds.staircase_dc
    stair_mask = (sf_mask | dc_mask) #total de perfis
    mixed_mask = (sf_mask & dc_mask)

    n_sf = int(sf_mask.sum())
    n_dc = int(dc_mask.sum())
    n_mixed = int(mixed_mask.sum())
    n_stair = int(stair_mask.sum())

    total_profiles_sf += n_sf
    total_profiles_dc += n_dc
    total_profiles_mixed += n_mixed
    total_profiles_stair += n_stair

    # ---------------- ESTATÍSTICAS VERTICAIS ----------------

    for i in range(n_region):

        if sf_mask.values[i]:

            ml_p = ds.ml_p.isel(Nobs=i).values
            ml_h = ds.ml_h.isel(Nobs=i).values

            valid = (
                ~np.isnan(ml_h) &
                ~np.isnan(ml_p) &
                (ml_h > 0)
            )

            all_ml_depth_sf.extend(ml_p[valid]) #adiciona elemento por elemento a lista
            all_ml_thickness_sf.extend(ml_h[valid])
            n_layers_sf.append(np.sum(valid)) #dá p fazer número médio de camadas por perfil SF

        if dc_mask.values[i]:

            ml_p = ds.ml_p.isel(Nobs=i).values
            ml_h = ds.ml_h.isel(Nobs=i).values

            valid = (
                ~np.isnan(ml_h) &
                ~np.isnan(ml_p) &
                (ml_h > 0)
            )

            all_ml_depth_dc.extend(ml_p[valid])
            all_ml_thickness_dc.extend(ml_h[valid])
            n_layers_dc.append(np.sum(valid))

    # ---------------- PONTOS ESPACIAIS ----------------

    lat = ds.lat.values
    lon = ds.lon.values

    lat_round = np.round(lat, 2)
    lon_round = np.round(lon, 2)

    for la, lo in zip(lat_round, lon_round):
        all_points.append((la, lo))

    for la, lo in zip(lat_round[stair_mask.values],
                      lon_round[stair_mask.values]):
        all_points_stair.append((la, lo))

    for la, lo in zip(lat_round[sf_mask.values],
                      lon_round[sf_mask.values]):
        all_points_sf.append((la, lo))

    for la, lo in zip(lat_round[dc_mask.values],
                      lon_round[dc_mask.values]):
        all_points_dc.append((la, lo))

    for la, lo in zip(lat_round[mixed_mask.values],
                      lon_round[mixed_mask.values]):
        all_points_mixed.append((la, lo))

    subset = ds[[
        "lat",
        "lon",
        "juld",
        "staircase_sf",
        "staircase_dc"
    ]]

    results.append(subset)


# =========================================================
# RESULTADOS FINAIS
# =========================================================

unique_points = set(all_points)
unique_points_stair = set(all_points_stair)
unique_points_sf = set(all_points_sf)
unique_points_dc = set(all_points_dc)
unique_points_mixed = set(all_points_mixed)

print("\n================ RESULTADOS ==================")

print("\n----- PERFIS -----")
print("Perfis globais:", total_profiles)
print("Perfis na região:", total_profiles_region)
print("Perfis com escada:", total_profiles_stair)
print("Perfis SF:", total_profiles_sf)
print("Perfis DC:", total_profiles_dc)
print("Perfis mistos:", total_profiles_mixed)

if total_profiles_region > 0:
    print("\nFrequência total de escadas:",
          round(100 * total_profiles_stair / total_profiles_region, 2), "%")

print("\n----- PONTOS ÚNICOS -----")
print("Pontos únicos na região:", len(unique_points))
print("Pontos únicos com escada:", len(unique_points_stair))
print("Pontos únicos SF:", len(unique_points_sf))
print("Pontos únicos DC:", len(unique_points_dc))
print("Pontos únicos mistos:", len(unique_points_mixed))

if len(unique_points) > 0:
    freq_spatial = len(unique_points_stair) / len(unique_points)
    print("Frequência espacial total:",
          round(freq_spatial * 100, 2), "%")

# =========================================================
# ESTATÍSTICAS VERTICAIS
# =========================================================

print("\n----- ESTATÍSTICAS VERTICAIS -----")

if len(all_ml_depth_sf) > 0:
    print("Profundidade média ML SF:",
          round(np.nanmean(all_ml_depth_sf), 1), "dbar")

if len(all_ml_depth_dc) > 0:
    print("Profundidade média ML DC:",
          round(np.nanmean(all_ml_depth_dc), 1), "dbar")

if len(all_ml_thickness_sf) > 0:
    print("Espessura média ML SF:",
          round(np.mean(all_ml_thickness_sf), 2), "dbar")

if len(all_ml_thickness_dc) > 0:
    print("Espessura média ML DC:",
          round(np.mean(all_ml_thickness_dc), 2), "dbar")

if len(n_layers_sf) > 0:
    print("Número médio de camadas SF:",
          round(np.mean(n_layers_sf), 2))

if len(n_layers_dc) > 0:
    print("Número médio de camadas DC:",
          round(np.mean(n_layers_dc), 2))


# =========================================================
# SALVAR DATASET FINAL
# =========================================================

if results:
    final_ds = xr.concat(results, dim="Nobs")
    final_ds.to_netcdf("staircases_subset_region.nc")


Processando: argo_00000000_00000249.nc
Perfis globais: 4
Perfis na região: 0

Processando: argo_00000250_00000499.nc
Perfis globais: 40
Perfis na região: 0

Processando: argo_00000500_00000749.nc
Perfis globais: 827
Perfis na região: 0

Processando: argo_00000750_00000999.nc
Perfis globais: 367
Perfis na região: 22

================ DIAGNÓSTICO ESPACIAL ================
Total de perfis analisados: 22

--- Valores inválidos ---
Lat NaN: 0
Lon NaN: 0
Lat Inf: 0
Lon Inf: 0

--- Faixa de valores ---
Lat min/max: 7.151 / 8.412
Lon min/max: -28.666 / -26.583
🌍 Longitude parece estar em -180–180

--- Repetição exata ---
Total pontos: 22
Pontos únicos: 22
Repetidos exatos: 0

--- Resolução espacial ---
Menor Δlat: 0.00399971
Mediana Δlat: 0.032000065
Menor Δlon: 0.00399971
Mediana Δlon: 0.070999146

--- Saltos espaciais entre perfis consecutivos ---
Saltos >10° encontrados: 0

--- Sensibilidade ao arredondamento ---
4 casas decimais → 22 pontos únicos
3 casas decimais → 22 pontos únicos
2 cas

In [11]:
#poontos
only_sf = len(unique_points_sf) - len(unique_points_mixed)
only_dc = len(unique_points_dc) - len(unique_points_mixed)

total_union = len(unique_points_sf | unique_points_dc)

print("Apenas SF:", only_sf, round(100 * only_sf / total_union, 2), "%")
print("Apenas DC:", only_dc, round(100 * only_dc / total_union, 2), "%")
print("Mistos:", len(unique_points_mixed), round(100 * len(unique_points_mixed) / total_union, 2), "%")

Apenas SF: 6918 39.67 %
Apenas DC: 3615 20.73 %
Mistos: 6910 39.62 %
